https://www.kaggle.com/datasets/shivamb/machine-predictive-maintenance-classification/data

Aplicar un docker!

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

60000 PIEZAS AL MES

SON 3000 AL DIA
CON UN COSTO 1500 PERO SI ES NECESARIO TERCIARIZAR PASAR A SER DE 4500 - Al Costo planificado se le suman 3000 pesos
$9.000.000 en perdidas en el dia en caso de parar

Son 10000 registros , suponiendo que fueron en el año

Cada paro requiere 1 dia (9 horas) de trabajo Perdida 9.000.000

Cada preventivo requiere 1 hora de trabajo Ganancia 8.000.000

Segun el dataset hubo 343 paros, representa una perdida de 3.087.000.000 anual

NO CONSIDERO GASTOS EN REPUESTOS Y MANO DE OBRA!!! A fin de simplificar la cuenta

In [2]:
# Definir una función para asignar puntajes según las condiciones
def asignar_puntajes(real, predicho):
    if real == 0 and predicho == 1: #APLICAS UN MANTENIMIENTO INNECESARIO
        return -1000000
    elif real == 1 and predicho == 1: #APLICAS UN MANTENIMIENTO CORRECTO
        return 8000000
    elif real == 0 and predicho == 0: #NO SE HACE NADA
        return 0
    elif real == 1 and predicho == 0: #ERA NECESARIO UN MANTENIMIENTO
        return -9000000
    else:
        return 0

In [4]:
df = pd.read_csv('predictive_maintenance.csv')
df.shape

(10000, 10)

**FEATURE ENGINEERING**

In [6]:
df.head(3)

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure


In [7]:
df = pd.DataFrame(df)

In [8]:
df = df.drop(['Product ID'], axis=1)
df = df.drop(['UDI'], axis = 1)
type_dummie = pd.get_dummies(df['Type'])
df2 = pd.concat([df, type_dummie], axis=1)    
df2.drop(['Type'], inplace=True, axis=1)
Failure_dummie = pd.get_dummies(df['Failure Type'])
df3 = pd.concat([df2, Failure_dummie], axis=1)    
df = df3.drop(['Failure Type'], axis = 1)

**FeatureTools**

In [10]:
import featuretools as ft

In [11]:
es = ft.EntitySet("maintenance")

es = es.add_dataframe(
    dataframe_name="maintenance",
    dataframe=df,
    index="UDI"#,
    #time_index="YrSold"#,
    #logical_types={
     #   "product_id": Categorical,
      #  "zip_code": PostalCode,
    #},
)

C:\Users\djoglar\AppData\Roaming\Python\Python311\site-packages\featuretools\entityset\entityset.py:1914: UserWarning: index UDI not found in dataframe, creating new integer column
  warnings.warn(


In [12]:
es

Entityset: maintenance
  DataFrames:
    maintenance [Rows: 10000, Columns: 16]
  Relationships:
    No relationships

In [13]:
feature_matrix, feature_defs = ft.dfs(entityset=es,
                                      target_dataframe_name="maintenance",
                                      max_depth=1)

In [14]:
feature_defs

[<Feature: Air temperature [K]>,
 <Feature: Process temperature [K]>,
 <Feature: Rotational speed [rpm]>,
 <Feature: Torque [Nm]>,
 <Feature: Tool wear [min]>,
 <Feature: Target>,
 <Feature: H>,
 <Feature: L>,
 <Feature: M>,
 <Feature: Heat Dissipation Failure>,
 <Feature: No Failure>,
 <Feature: Overstrain Failure>,
 <Feature: Power Failure>,
 <Feature: Random Failures>,
 <Feature: Tool Wear Failure>]

In [15]:
feature_matrix

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,H,L,M,Heat Dissipation Failure,No Failure,Overstrain Failure,Power Failure,Random Failures,Tool Wear Failure
UDI,,,,,,,,,,,,,,,
0,298.1,308.6,1551,42.8,0,0,False,False,True,False,True,False,False,False,False
1,298.2,308.7,1408,46.3,3,0,False,True,False,False,True,False,False,False,False
2,298.1,308.5,1498,49.4,5,0,False,True,False,False,True,False,False,False,False
3,298.2,308.6,1433,39.5,7,0,False,True,False,False,True,False,False,False,False
4,298.2,308.7,1408,40.0,9,0,False,True,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,298.8,308.4,1604,29.5,14,0,False,False,True,False,True,False,False,False,False
9996,298.9,308.4,1632,31.8,17,0,True,False,False,False,True,False,False,False,False
9997,299.0,308.6,1645,33.4,22,0,False,False,True,False,True,False,False,False,False


In [16]:
#ft.list_primitives()

No crea ningun campo nuevo!

problemas encontrados con featureTools:

-Al no tener fecha el dataframe, no genera ningun campo relacionado a fecha

-Al no tener otras tablas para relacionar no se puede explotar mas lo datos


In [17]:
#Preparacion de datos

In [18]:
X = df.drop(['Target'], axis=1)
y = df.Target
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25, random_state=42) #

AutoFeat

In [22]:
from autofeat import AutoFeatClassifier
#Con estos parametros me muestra los pasos y le seteo la cantidad de pasos
model = AutoFeatClassifier(verbose=1, feateng_steps=1)
new_df = model.fit_transform(X, y) #Esto tarda!

2024-05-11 15:28:04,495 INFO: [AutoFeat] The 1 step feature engineering process could generate up to 105 features.
2024-05-11 15:28:04,497 INFO: [AutoFeat] With 10000 data points this new feature matrix would use about 0.00 gb of space.
2024-05-11 15:28:04,499 INFO: [feateng] Step 1: transformation of original features


2024-05-11 15:28:05,846 INFO: [feateng] Generated 26 transformed features from 15 original features - done.
2024-05-11 15:28:05,862 INFO: [feateng] Generated altogether 28 new features in 1 steps
2024-05-11 15:28:05,862 INFO: [feateng] Removing correlated features, as well as additions at the highest level
2024-05-11 15:28:05,873 INFO: [feateng] Generated a total of 4 additional features
2024-05-11 15:28:05,880 INFO: [featsel] Feature selection run 1/5


[featsel] Scaling data...done.


2024-05-11 15:28:31,153 INFO: [featsel] Feature selection run 2/5
2024-05-11 15:28:55,068 INFO: [featsel] Feature selection run 3/5
2024-05-11 15:29:19,158 INFO: [featsel] Feature selection run 4/5
2024-05-11 15:29:45,167 INFO: [featsel] Feature selection run 5/5
2024-05-11 15:30:11,905 INFO: [featsel] 3 features after 5 feature selection runs
2024-05-11 15:30:11,905 INFO: [featsel] 3 features after correlation filtering
2024-05-11 15:30:17,712 INFO: [featsel] 2 features after noise filtering
2024-05-11 15:30:17,712 INFO: [AutoFeat] Final dataframe with 15 feature columns (0 new).
2024-05-11 15:30:17,712 INFO: [AutoFeat] Training final classification model.
2024-05-11 15:30:17,967 INFO: [AutoFeat] Trained model: largest coefficients:
2024-05-11 15:30:17,967 INFO: [6.44481391]
2024-05-11 15:30:17,967 INFO: 10.040245 * No Failure
2024-05-11 15:30:17,967 INFO: 7.072462 * Random Failures
2024-05-11 15:30:17,975 INFO: [AutoFeat] Final score: 0.9991


In [23]:
new_df.head(3)

,UDI,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],H,L,M,Heat Dissipation Failure,No Failure,Overstrain Failure,Power Failure,Random Failures,Tool Wear Failure
0,0.0,298.1,308.6,1551.0,42.8,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,298.2,308.7,1408.0,46.3,3.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2.0,298.1,308.5,1498.0,49.4,5.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [24]:
df_test = model.transform(X_test)
print("Final R^2: %.4f" % model.score(new_df, y))
#y_pred = model.predict(X_test)

Final R^2: 0.9991


In [25]:
X_train = model.transform(X_train)

In [26]:
X_test = model.transform(X_test)

Creacion de variables manuales

In [27]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(7500, 15)
(2500, 15)
(7500,)
(2500,)


In [28]:
###FIN FEATURE

*****MODELS*****

**TPOT**

In [176]:
# Importo las librerias
from tpot import TPOTClassifier

Defino como metrica el Recall ya que me interesa que detecte mas a los casos positivos, aunque de falsos positivos

In [177]:
generations=2
population_size=10
pipeline_optimizer = TPOTClassifier(
    generations=generations,
    population_size=population_size,
    verbosity=2,
    random_state=42,
    memory='auto',
    n_jobs=-1,
    scoring='recall') #Me interesa que ataque a los positivos

In [178]:
%%time
pipeline_optimizer.fit(X_train, y_train)

Optimization Progress:   0%|          | 0/30 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9737945492662474

Generation 2 - Current best internal CV score: 0.9737945492662474

Best pipeline: GaussianNB(input_matrix)
CPU times: total: 3.23 s
Wall time: 56.2 s


TPOTClassifier(generations=2, memory='auto', n_jobs=-1, population_size=10,
               random_state=42, scoring='recall', verbosity=2)

In [179]:
pipeline_optimizer.export('tpot_export_default_pipeline.py')

In [180]:
y_pred_tpot = pipeline_optimizer.predict(X_test)

In [183]:
df_resultados_tpot = pd.DataFrame({'Real': y_test, 'Predicho': y_pred_tpot})

      Real  Predicho
6252     0         0
4684     1         0
1731     0         0
4742     0         0
4521     0         0
...    ...       ...
4862     0         0
7025     0         0
7647     0         0
7161     0         0
73       0         0

[2500 rows x 2 columns]


In [184]:
# Aplicar la función a cada fila del DataFrame para crear la nueva columna
df_resultados_tpot['Puntaje'] = df_resultados_tpot.apply(lambda row: asignar_puntajes(row['Real'], row['Predicho']), axis=1)

# Imprimir el DataFrame con la nueva columna
print(df_resultados_tpot)

      Real  Predicho  Puntaje
6252     0         0        0
4684     1         0 -9000000
1731     0         0        0
4742     0         0        0
4521     0         0        0
...    ...       ...      ...
4862     0         0        0
7025     0         0        0
7647     0         0        0
7161     0         0        0
73       0         0        0

[2500 rows x 3 columns]


In [187]:
GANANCIA_FINAL_tpot = df_resultados_tpot['Puntaje'].sum()
GANANCIA_FINAL_tpot

536000000

In [186]:
print("Recall en el conjunto de prueba:", pipeline_optimizer.score(X_test, y_test))

Recall en el conjunto de prueba: 0.9722222222222222


In [41]:
recall_tpot=pipeline_optimizer.score(X_test, y_test)

**H20**

In [42]:
import h2o
from h2o.automl import H2OAutoML

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 21+35-LTS-2513, mixed mode, sharing)
  Starting server from C:\Users\djoglar\AppData\Roaming\Python\Python311\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\djoglar\AppData\Local\Temp\tmp2jmtudeq
  JVM stdout: C:\Users\djoglar\AppData\Local\Temp\tmp2jmtudeq\h2o_djoglar_started_from_python.out
  JVM stderr: C:\Users\djoglar\AppData\Local\Temp\tmp2jmtudeq\h2o_djoglar_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,America/Argentina/Buenos_Aires
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,1 month and 28 days
H2O_cluster_name:,H2O_from_python_djoglar_ar4oot
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,5.910 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [ ]:
# Convierto los dataframe de pandas a dataframe de H20
msk = np.random.rand(len(new_df)) < 0.25
df_train_H2O = new_df[msk]
df_test_H2O = new_df[~msk]

In [ ]:
# Convierto los dataframe de pandas a dataframe de H2O
train_H2O = h2o.H2OFrame(df_train_H2O)
test_H2O = h2o.H2OFrame(df_test_H2O)

In [ ]:
# Defino cuales columnas son las variables de entrenamiento y cual es el target.
x_H2O = train_H2O.columns
y_H2O = "Target"
x_H2O.remove(y_H2O)
print(x_H2O)

In [ ]:
x_H2O

In [ ]:
# For binary classification, response should be a factor
train_H2O[y_H2O] = train_H2O[y_H2O].asfactor()
test_H2O[y_H2O] = test_H2O[y_H2O].asfactor()

In [ ]:
# Entreno el modelo
aml = H2OAutoML(max_models=20, seed=1, max_runtime_secs=60*2, sort_metric = "AUCPR")
aml.train(x=x_H2O, y=y_H2O, training_frame=train_H2O)

In [58]:
#Convierto la serie a dataframe
y_test_2 = pd.DataFrame(y_test)
y_train_2 = pd.DataFrame(y_train)

# Inicia H2O
h2o.init()

# Convierte tus datos de entrenamiento a objetos H2OFrame
x_train_h2o = h2o.H2OFrame(X_train)
y_train_h2o = h2o.H2OFrame(y_train_2)

# Combina los datos de entrada con la variable objetivo
train_h2o = x_train_h2o.cbind(y_train_h2o)

y_H2O = "Target"

train_h2o[y_H2O] = train_h2o[y_H2O].asfactor()

# Define y configura el objeto H2OAutoML
aml = H2OAutoML(max_models=20, seed=1, max_runtime_secs=60*2, sort_metric="AUCPR")

# Entrena el modelo
aml.train(x=x_train_h2o.names, y=y_train_h2o.names[0], training_frame=train_h2o)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 hour 1 min
H2O_cluster_timezone:,America/Argentina/Buenos_Aires
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,1 month and 28 days
H2O_cluster_name:,H2O_from_python_djoglar_ar4oot
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,5.876 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |█
16:36:06.931: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_1_AutoML_2_20240511_163606


Status of Neuron Layers: predicting Target, 2-class classification, bernoulli distribution, CrossEntropy loss, 402 weights/biases, 10,9 KB, 78.230 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate            rate_rms             momentum    mean_weight            weight_rms           mean_bias             bias_rms
--  -------  -------  ---------  ---------  ----  ----  -------------------  -------------------  ----------  ---------------------  -------------------  --------------------  --------------------
    1        15       Input      0.0
    2        10       Rectifier  0.0        0.0   0.0   0.33308052185651227  0.41402697563171387  0.0         -0.03244120010155408   0.30071139335632324  0.5925587755932604    0.1612347960472107
    3        10       Rectifier  0.0        0.0   0.0   0.4093049014650751   0.4536106586456299   0.0         0.007062344754085643   0.32272040843963623  0.9729041494223842    0.0685892105102539
    4        10       Rectifier  0.0        0.0   0.0   0.37739588008844294  0.46133458614349365  0.0         -0.004272151961922646  0.3306940793991089   0.9951901201554161    0.048144906759262085
    5        2        Softmax               0.0   0.0   0.28615088590886445  0.41376805305480957  0.0         -0.25259336438030006   1.4731340408325195   0.005092308914847854  0.032778412103652954

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.0009333219522306571
RMSE: 0.030550318365455003
LogLoss: 0.012108679147685897
Mean Per-Class Error: 0.013108614232209739
AUC: 0.9913098568721905
AUCPR: 0.9810107747452073
Gini: 0.9826197137443811

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9999045839859961
       0     1    Error    Rate
-----  ----  ---  -------  ------------
0      7233  0    0        (0.0/7233.0)
1      7     260  0.0262   (7.0/267.0)
Total  7240  260  0.0009   (7.0/7500.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.999905     0.986717  51
max f2                       0.999905     0.978916  51
max f0point5                 0.999905     0.994644  51
max accuracy                 0.999905     0.999067  51
max precision                1            1         0
max recall                   6.95982e-08  1         396
max specificity              1            1         0
max absolute_mcc             0.999905     0.986327  51
max min_per_class_accuracy   1.49437e-05  0.980783  129
max mean_per_class_accuracy  0.999905     0.986891  51
max tns                      1            7233      0
max fns                      1            251       0
max fps                      1.27244e-08  7233      399
max tps                      6.95982e-08  267       396
max tnr                      1            1         0
max fnr                      1            0.940075  0
max fpr                      1.27244e-08  1         399
max tpr                      6.95982e-08  1         396

Gains/Lift Table: Avg response rate:  3,56 %, avg score:  3,47 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.01                        1                  28.0899    28.0899            1                1            1                           1                   0.280899        0.280899                   2708.

In [59]:
# Ranking de los modelos entrenados
lb = aml.leaderboard
lb.head()

model_id,aucpr,auc,logloss,mean_per_class_error,rmse,mse
DeepLearning_1_AutoML_2_20240511_163606,0.980013,0.991942,0.0113108,0.0131086,0.04524,0.00204666
GBM_grid_1_AutoML_2_20240511_163606_model_2,0.978351,0.991137,0.00770052,0.0131086,0.0308923,0.000954332
GBM_4_AutoML_2_20240511_163606,0.977138,0.98961,0.00879343,0.0131086,0.0325679,0.00106067
GBM_3_AutoML_2_20240511_163606,0.977101,0.990774,0.0085347,0.0131086,0.0311491,0.000970268
GBM_grid_1_AutoML_2_20240511_163606_model_1,0.977004,0.990114,0.00958485,0.0131086,0.0338989,0.00114914
GBM_grid_1_AutoML_2_20240511_163606_model_4,0.976755,0.990401,0.0125035,0.0151887,0.0493486,0.00243528
GBM_2_AutoML_2_20240511_163606,0.976677,0.989131,0.00848383,0.0131086,0.0312791,0.000978385
GBM_grid_1_AutoML_2_20240511_163606_model_5,0.976485,0.98891,0.00885944,0.0131086,0.0310829,0.000966146
DRF_1_AutoML_2_20240511_163606,0.976457,0.98714,0.0280675,0.0131086,0.0337241,0.00113732
GBM_5_AutoML_2_20240511_163606,0.976453,0.988663,0.00805416,0.0131086,0.0306628,0.000940206


In [ ]:
# Descripcion de todos los modelos entrenados
#exa = aml.explain(test_H2O)

In [60]:
best_model = aml.leader

In [62]:
# Convierte tus datos de entrenamiento a objetos H2OFrame
x_test_h2o = h2o.H2OFrame(X_test)
y_test_h2o = h2o.H2OFrame(y_test_2)

# Combina los datos de entrada con la variable objetivo
test_h2o = x_test_h2o.cbind(y_test_h2o)

y_H2O = "Target"

test_h2o[y_H2O] = test_h2o[y_H2O].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [78]:
top_models = aml.leaderboard.as_data_frame().head(3)['model_id'].values

C:\Users\djoglar\AppData\Roaming\Python\Python311\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [135]:
# Crear una lista para almacenar los DataFrames de cada modelo
results_dfs = []

# Iterar sobre los modelos
for model_id in top_models:
    model = h2o.get_model(model_id)
    predictions = model.predict(test_h2o)
    y_pred = predictions[0].as_data_frame().values.flatten()
    
    # Obtener los valores reales de y_test
    y_real = y_test.values
    
    # Aplicar la función de ganancia a las predicciones de cada modelo
    puntajes = [asignar_puntajes(y_real[i], y_pred[i]) for i in range(len(y_real))]
    
    # Crear un DataFrame para el modelo actual
    df = pd.DataFrame({"Valor_Predicho": y_pred, "Valor_Real": y_real, "Puntaje": puntajes})
    
    # Agregar el nombre del modelo como una columna en el DataFrame
    df['Modelo'] = model_id
    
    results_dfs.append(df)

deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%


C:\Users\djoglar\AppData\Roaming\Python\Python311\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


C:\Users\djoglar\AppData\Roaming\Python\Python311\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


C:\Users\djoglar\AppData\Roaming\Python\Python311\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


      Valor_Predicho  Valor_Real  Puntaje  \
0                  0           0        0   
1                  0           1 -9000000   
2                  0           0        0   
3                  0           0        0   
4                  0           0        0   
...              ...         ...      ...   
2495               0           0        0   
2496               0           0        0   
2497               0           0        0   
2498               0           0        0   
2499               0           0        0   

                                       Modelo  
0     DeepLearning_1_AutoML_2_20240511_163606  
1     DeepLearning_1_AutoML_2_20240511_163606  
2     DeepLearning_1_AutoML_2_20240511_163606  
3     DeepLearning_1_AutoML_2_20240511_163606  
4     DeepLearning_1_AutoML_2_20240511_163606  
...                                       ...  
2495  DeepLearning_1_AutoML_2_20240511_163606  
2496  DeepLearning_1_AutoML_2_20240511_163606  
2497  DeepLearning_1_AutoML

In [138]:
#results_df['Predicho'] = results_df.apply(lambda row: 1 if row['Predicciones'] > 0.80 else 0, axis=1)
#results_dfs['Puntaje'] = results_dfs.apply(lambda row: asignar_puntajes(row['Valor_Real'], row['Valor_Predicho']), axis=1)

results_dfs = pd.concat(results_dfs, ignore_index=True)
results_dfs

,Valor_Predicho,Valor_Real,Puntaje,Modelo
0,0,0,0,DeepLearning_1_AutoML_2_20240511_163606
1,0,1,-9000000,DeepLearning_1_AutoML_2_20240511_163606
2,0,0,0,DeepLearning_1_AutoML_2_20240511_163606
3,0,0,0,DeepLearning_1_AutoML_2_20240511_163606
4,0,0,0,DeepLearning_1_AutoML_2_20240511_163606
...,...,...,...,...
7495,0,0,0,GBM_4_AutoML_2_20240511_163606
7496,0,0,0,GBM_4_AutoML_2_20240511_163606
7497,0,0,0,GBM_4_AutoML_2_20240511_163606
7498,0,0,0,GBM_4_AutoML_2_20240511_163606


In [214]:
GANANCIAS_H2O = results_dfs.groupby('Modelo')['Puntaje'].sum()

In [215]:
GANANCIAS_H2O = pd.DataFrame(GANANCIAS_H2O)
GANANCIAS_H2O.reset_index(inplace=True)

In [216]:
GANANCIAS_H2O.rename(columns={'Puntaje': 'GANANCIA'}, inplace=True)
GANANCIAS_H2O

,Modelo,GANANCIA
0,DeepLearning_1_AutoML_2_20240511_163606,542000000
1,GBM_4_AutoML_2_20240511_163606,542000000
2,GBM_grid_1_AutoML_2_20240511_163606_model_2,542000000


In [ ]:
#######################################################

##Probar CatBoost Classifier!!
https://www.kaggle.com/code/prashant111/catboost-classifier-in-python/notebook

In [150]:
from catboost import CatBoostClassifier
from sklearn.metrics import recall_score

# Dividir los datos en conjunto de entrenamiento y prueba

# Crear un clasificador CatBoost
cat_model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, loss_function='Logloss', custom_metric = ['Recall'], random_seed=42)

# Entrenar el modelo
cat_model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=500, verbose=100)

# Hacer predicciones en el conjunto de prueba
y_pred_cat = cat_model.predict(X_test)

recall_cat = recall_score(y_test, y_pred_cat)
print("Recall del modelo:", recall_cat)

0:	learn: 0.4183260	test: 0.4174245	best: 0.4174245 (0)	total: 9.5ms	remaining: 9.49s
100:	learn: 0.0025550	test: 0.0060341	best: 0.0060319 (98)	total: 932ms	remaining: 8.3s
200:	learn: 0.0008178	test: 0.0059905	best: 0.0058416 (130)	total: 1.84s	remaining: 7.32s
300:	learn: 0.0004620	test: 0.0062624	best: 0.0058416 (130)	total: 2.78s	remaining: 6.45s
400:	learn: 0.0003075	test: 0.0065888	best: 0.0058416 (130)	total: 3.76s	remaining: 5.61s
500:	learn: 0.0002342	test: 0.0068293	best: 0.0058416 (130)	total: 4.64s	remaining: 4.62s
600:	learn: 0.0001903	test: 0.0070265	best: 0.0058416 (130)	total: 5.71s	remaining: 3.79s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.005841626977
bestIteration = 130

Shrink model to first 131 iterations.
Recall del modelo: 0.9722222222222222


In [154]:
# Crear un DataFrame con las predicciones del modelo y los valores reales
results_cat = pd.DataFrame({'Prediccion': y_pred_cat, 'Valor_Real': y_test})

# Aplicar la función de ganancia a las predicciones y calcular el puntaje final
results_cat['Puntaje'] = results_cat.apply(lambda row: asignar_puntajes(row['Valor_Real'], row['Prediccion']), axis=1)

# Mostrar el DataFrame resultante
GANANCIAS_cat = results_cat['Puntaje'].sum()
GANANCIAS_cat

542000000

**METRICAS FINALES**

*RECALL*

In [168]:
#METRICAS H2O
recall_results=[]
for model_id in top_models:
    model = h2o.get_model(model_id)
    predictions = model.predict(test_h2o)
    y_pred = predictions[0].as_data_frame().values.flatten()
    y_real = y_test.values
    recall = recall_score(y_real, y_pred)
    recall_results.append({'Modelo': model_id, 'Recall': recall})

recall_df_final = pd.DataFrame(recall_results)

deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%


C:\Users\djoglar\AppData\Roaming\Python\Python311\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


C:\Users\djoglar\AppData\Roaming\Python\Python311\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


C:\Users\djoglar\AppData\Roaming\Python\Python311\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [170]:
others_score =pd.DataFrame ({'Modelo': ['Modelo Tpot', 'Modelo CAT'], 'Recall': [recall_tpot, recall_cat]})  
recall_df_final = pd.concat([recall_df_final,others_score])

In [171]:
recall_df_final

,Modelo,Recall
0,DeepLearning_1_AutoML_2_20240511_163606,0.972222
1,GBM_grid_1_AutoML_2_20240511_163606_model_2,0.972222
2,GBM_4_AutoML_2_20240511_163606,0.972222
0,Modelo Tpot,0.972222
1,Modelo CAT,0.972222


*AHORRO POR MODELO*

In [217]:
# Agregar una nueva fila al DataFrame
ahorro_models =pd.DataFrame ({'Modelo': ['Sin prediccion', 'Modelo Tpot', 'Modelo CAT'], 'GANANCIA': [y_test_2['Target'].sum()*3000*3000*-1, GANANCIA_FINAL_tpot, GANANCIAS_cat]})  # Agrega los valores que desees para la nueva fila
Ahorro_modelo_final= pd.concat([ahorro_models,GANANCIAS_H2O])

In [218]:
Ahorro_modelo_final

,Modelo,GANANCIA
0,Sin prediccion,-648000000
1,Modelo Tpot,536000000
2,Modelo CAT,542000000
0,DeepLearning_1_AutoML_2_20240511_163606,542000000
1,GBM_4_AutoML_2_20240511_163606,542000000
2,GBM_grid_1_AutoML_2_20240511_163606_model_2,542000000
